<a href="https://colab.research.google.com/github/Sai145/2048/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
!pip install -q findspark


In [4]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 198 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=439812ccc6f4243599c09268816c3fedc46c3106666803e68cd24e58cebe3a31
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("test")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [8]:
spark

In [20]:
from pyspark.sql import *
from datetime import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [59]:
campaigns=spark.read.option("multiline","true").json("/content/drive/MyDrive/Data/data1.json")
Users=spark.read.option("multiline","true").json("/content/drive/MyDrive/Data/data2.json")
Stores=spark.read.option("multiline","true").json("/content/drive/MyDrive/Data/data3.json")

campaigns=campaigns.withColumn("date",to_date(col("event_time"))).withColumn("hour",hour(col("event_time"))
).withColumn("value",lit(col("os_type"))).withColumn("type",lit("os_type"))

Users=Users.withColumnRenamed("gender","value").withColumn("type",lit("gender"))

Stores=Stores.withColumn("Place_id",explode(col("place_ids")))
Stores=Stores.withColumnRenamed("store_name","value").withColumn("type",lit("storename"))

In [60]:


res1=campaigns.groupBy("campaign_id","date","hour","type","value","event_type").count()
res1.show()



+-----------+----------+----+-------+-------+----------+-----+
|campaign_id|      date|hour|   type|  value|event_type|count|
+-----------+----------+----+-------+-------+----------+-----+
|    ABCDFAE|2018-10-12|  13|os_type|android|     click|    1|
|    ABCDFAE|2018-10-12|  13|os_type|android|impression|    1|
|    ABCDFAE|2018-10-12|  13|os_type|    ios|impression|    1|
|    ABCDFAE|2018-10-12|  13|os_type|android|  video ad|    1|
+-----------+----------+----+-------+-------+----------+-----+



In [61]:
res2=campaigns.join(Stores,campaigns.place_id==Stores.Place_id,"inner")
df2=res2.groupBy("campaign_id","date","hour",Stores.type,Stores.value,"event_type").count()
df2.show()



+-----------+----------+----+---------+-------------+----------+-----+
|campaign_id|      date|hour|     type|        value|event_type|count|
+-----------+----------+----+---------+-------------+----------+-----+
|    ABCDFAE|2018-10-12|  13|storename|     McDonald|impression|    2|
|    ABCDFAE|2018-10-12|  13|storename|   BurgerKing|     click|    1|
|    ABCDFAE|2018-10-12|  13|storename|     McDonald|     click|    1|
|    ABCDFAE|2018-10-12|  13|storename|   BurgerKing|impression|    1|
|    ABCDFAE|2018-10-12|  13|storename|shoppers stop|  video ad|    1|
+-----------+----------+----+---------+-------------+----------+-----+



In [62]:
res3=campaigns.join(Users,campaigns.user_id==Users.user_id,"inner")
df3=res3.groupBy("campaign_id","date","hour",Users.type,Users.value,"event_type").count()
df3.show()

+-----------+----------+----+------+------+----------+-----+
|campaign_id|      date|hour|  type| value|event_type|count|
+-----------+----------+----+------+------+----------+-----+
|    ABCDFAE|2018-10-12|  13|gender|  male|     click|    1|
|    ABCDFAE|2018-10-12|  13|gender|female|impression|    1|
|    ABCDFAE|2018-10-12|  13|gender|  male|  video ad|    1|
|    ABCDFAE|2018-10-12|  13|gender|  male|impression|    1|
+-----------+----------+----+------+------+----------+-----+

